In [ ]:
from code_map import data_handling, utils
import pickle


In [ ]:
#return L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./one_week_collections.pkl")

In [ ]:
compatible_list = utils.get_compatibility_dict(L, M)

In [ ]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

In [ ]:
aggregated_results = utils.run_batched_optimization_model(L, M, H, F, P_h_m, Vp_h_m, Vm_m, R_m, R_h_l, Fu_h_l, Fd_h_l, compatible_list, "batched_week_v3.log", "batched_model")

In [ ]:
for b in range(7):
    # Determine the subset of hours for this batch
    start_index = b * 24
    end_index = min((b + 1) * 24, len(H))
    batch_H = H[start_index:end_index]
    print(f"Batch {b} : {start_index} - {end_index}")

In [ ]:
one_week_model, x, y, w, d  = utils.run_optimization_model(L, M, H, F, Ir_hlm, Ia_hlm, Va_hm, Vp_h_m, Vm_m, R_m, R_h_l, Fu_h_l, Fd_h_l, compatible_list, "new_week_test.log", "test_model")

In [ ]:
utils.test_solution_validity(x, y, w, Va_hm, L, M, H, dominant_directions= dominant_directions, F = F)

In [ ]:
new_x_values = {(h, l, m): one_week_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}

In [ ]:
utils.compare_current_and_new_solution(current_x_pkl_file= "current_x_values_for_day.pkl", new_x_values= new_x_values, L= L, M= M, H= H, dominant_directions= dominant_directions)

In [ ]:

# Save these values
with open('current_x_values_for_week.pkl', 'wb') as f:
    pickle.dump(new_x_values, f)
                